In [1]:
from bs4 import BeautifulSoup
import glob
from tqdm import tqdm

In [2]:
def parse_slotech_post(post):
    return {
        'user': post.find('a', class_='avatar').get('title'),
        'time': post.find('time').get('datetime'),
        'text': post.find('div', class_='content').text,
    }

In [3]:
def parse_slotech_thread(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        h = f.read()
    soup = BeautifulSoup(h, 'html.parser')

    content = soup.find('div', id='content')

    h2 = content.find('h2')
    subforum = h2.find_all('span', {'itemprop': True})[-1].text
    title = content.find('h3').text

    posts = soup.find_all('div', class_='post')

    return {
        'file': filename,
        'subforum': subforum,
        'title': title,
        'data': [parse_slotech_post(post) for post in posts]
    }

In [13]:
threads = glob.glob('HTTRack/SloTech/slo-tech.com/forum/t*.html')
threads.extend(glob.glob('HTTRack/SloTech/slo-tech.com/forum/**/[123456789]*.html'))

In [14]:
len(threads)

60165

In [15]:
parsed = []

In [16]:
with tqdm(total=len(threads)) as pbar:
    for thread in threads:
        try:
            parsed.append(parse_slotech_thread(thread))
        except:
            pass
        pbar.update(1)

100%|███████████████████████████████████████████████████████████████████████████| 60165/60165 [2:37:58<00:00,  6.35it/s]


In [17]:
len(parsed)

58235

In [19]:
import json
with open('parsed_slo-tech.json', 'w', encoding='utf-8') as f:
    json.dump(parsed, f, ensure_ascii=False)